# Logistic Regression, SVM and DTree Classifer

In this we will demonstarte how to use Logistic Regression in scikit-learn to perfrom logistic regression

In this tutorial we will demonstrate how to use the SVM class in scikit-learn to perform logistic regression on a dataset.

In this tutorial we will demonstrate how to use the `DecisionTreeClassifer` class in `scikit-learn` and we use boosting techniques to perform classifications predictions


## 1 Setup
Import modules


In [14]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot as plt
from sklearn import preprocessing
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from scipy import stats

np.random.seed(1)

## 2 Load data
Load data (it's already cleaned and preprocessed)


In [15]:
# Uncomment the following snippet of code to debug problems with finding the .csv file path
# This snippet of code will exit the program and print the current working directory.
#import os
#print(os.getcwd())

In [16]:
X_train = pd.read_csv("thoracic_train_X_risk.csv")
X_test = pd.read_csv("thoracic_test_X_risk.csv")
y_train = pd.read_csv("thoracic_train_y_risk.csv")
y_test = pd.read_csv("thoracic_test_y_risk.csv")

First, let's create a dataframe to load the model performance metrics into.

In [17]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

## 3 Model the data

## Logistic regression using Randomsearch

In [18]:
score_measure = "accuracy"
kfolds = 5

param_grid = {'C':[0.001,0.01,0.1,1,10], # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(500,1000)
    
    
}

lr_rs = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = lr_rs, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train,y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestlr_rs = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.8601864801864803
... with parameters: {'solver': 'saga', 'penalty': 'l1', 'max_iter': 932, 'C': 0.1}


C:\Users\hiran\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
910 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\hiran\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hiran\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\hiran\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 64, in _check_solver
    raise ValueError(
V

In [19]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8297872 Precision=nan Recall=0.0000000 F1=0.0000000


C:\Users\hiran\AppData\Local\Temp\ipykernel_37296\520604890.py:6: RuntimeWarning: invalid value encountered in longlong_scalars
  print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")


## Logistic regression using grid search

In [22]:
score_measure = "accuracy"
kfolds = 5
best_penality = rand_search.best_params_['penalty']
best_solver = rand_search.best_params_['solver']
min_regulization_strength=rand_search.best_params_['C']
min_iter = rand_search.best_params_['max_iter']

#Using the best parameters from the Random Search to use as range for the parameters to do the grid search
param_grid = {
    
    'C':np.arange(min_regulization_strength-1,min_regulization_strength+1), 
               'penalty':[best_penality],
              'solver':[best_solver],
              'max_iter': np.arange(min_iter-400,min_iter+400)
}

lr_gs =  LogisticRegression()
grid_search = GridSearchCV(estimator = lr_gs, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                return_train_score=True)

_ = grid_search.fit(X_train,y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestlr_gs = grid_search.best_estimator_

Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
The best accuracy score is 0.848018648018648
... with parameters: {'C': 9, 'max_iter': 328, 'penalty': 'l1', 'solver': 'saga'}


C:\Users\hiran\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hiran\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [23]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8226950 Precision=0.0000000 Recall=0.0000000 F1=0.0000000


# SVM Classification by using Random Search CV

In [26]:
score_measure = "accuracy"
kfolds = 5

param_grid = {'C':np.arange(0.1,100,10),  #  regularization parameter.
               'kernel':['linear', 'rbf','poly'],
              'gamma':['scale','auto'],
              'degree':np.arange(1,10), #degree is for the polynomial kernal
              'coef0':np.arange(1,10) #coef0 is for the polynomial kernal
                  
}

svm_poly_model_rs = SVC()
rand_search = RandomizedSearchCV(estimator = svm_poly_model_rs, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestrecallsvm_rand = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.8601864801864803
... with parameters: {'kernel': 'poly', 'gamma': 'auto', 'degree': 1, 'coef0': 1, 'C': 90.1}


C:\Users\hiran\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [27]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8226950 Precision=0.0000000 Recall=0.0000000 F1=0.0000000


# SVM Classification by using Grid Search CV

In [30]:
score_measure = "accuracy"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-2,C+2),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

svm_model = SVC()
grid_search = GridSearchCV(estimator = svm_model, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecall = grid_search.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best accuracy score is 0.8054079254079255
... with parameters: {'C': 48.1, 'gamma': 'scale', 'kernel': 'poly'}


C:\Users\hiran\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.7730496 Precision=0.2142857 Recall=0.1250000 F1=0.1578947


# Decision tree by using Random search cv

In [34]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 50), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestdtree_rand = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.8601864801864803
... with parameters: {'min_samples_split': 48, 'min_samples_leaf': 31, 'min_impurity_decrease': 0.0041, 'max_leaf_nodes': 14, 'max_depth': 30, 'criterion': 'gini'}


In [35]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.7730496 Precision=0.2142857 Recall=0.1250000 F1=0.1578947


# Decision tree by using Grid search cv

In [38]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(30,36),  
    'min_samples_leaf': np.arange(6,12),
    'min_impurity_decrease': np.arange(0.0048, 0.0054, 0.0001),
    'max_leaf_nodes': np.arange(162,168), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

Dout_GS = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = Dout_GS, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestdtree_grid = grid_search.best_estimator_

Fitting 5 folds for each of 9072 candidates, totalling 45360 fits
The best accuracy score is 0.8601864801864803
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 162, 'min_impurity_decrease': 0.0048, 'min_samples_leaf': 7, 'min_samples_split': 30}


In [39]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
print(f"Accuracy={(TP+TN)/(TP+TN+FP+FN):.7f} Precision={TP/(TP+FP):.7f} Recall={TP/(TP+FN):.7f} F1={2*TP/(2*TP+FP+FN):.7f}")

Accuracy=0.8156028 Precision=0.2500000 Recall=0.0416667 F1=0.0714286


## Analysis

One of the often used evaluation metric to evaluate how well a model performs on a dataset is accuracy. When the dataset has a balanced distribution of classes, which means that each class appears in the dataset with nearly the same frequency, it is usually used and most of the data in my data set have balanced data and as a reason I have considered accuracy as a metric by dividing the number of samples that were properly categorised by the total sample count. It shows how often the model generated accurate predictions.

To fit the Thoracic Surgery data in this code, we used a variety of models, including logistic regression, SVM polynomial kernel, and decision tree models for both random and grid search. We then evaluated their performance using the metric accuracy. High accuracy score shows that the model represents the proportion of correct predictions made. In comparison of overall scores decision tree model for both random and grid search model outperformed the logistic regression and svm for random, and grid search models when we look into the accuracy scores of all other models, with a better accuracy score of 86.01%, even though both logistic and svm random search are also with 86.01% as mentioned previously decision tree has scored 86.01% score for both the searches and the accuracy values of logistic regression and svm grid search are standing at 84.80% and 80.54%,  showing that the decision tree for both random and grid search model fits best when compared to all the models for this dataset.